In [1]:
import os
import sys
import numpy as np
import time
# import pyaudio
import wave
import zipfile
import wavio
import random
import asyncio
import IPython
from queue import Queue
from threading import Thread
import argparse
import array
np.set_printoptions(threshold=sys.maxsize)

In [2]:
import sounddevice as sd
import soundfile as sf
import scipy

In [3]:
import tensorflow as tf

2024-05-20 15:07:53.040633: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-20 15:07:53.060773: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-20 15:07:53.060793: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-20 15:07:53.061320: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-20 15:07:53.064656: I tensorflow/core/platform/cpu_feature_guar

In [4]:
sys.path.append("../ipynb_files/")
sys.path.append("../")
# sys.path.append("../../src/SharedLibs/")

In [5]:
from common import utils as U
from TestSharedLib.bytes import to_bytes, from_bytes, byte_conversion_tests, load_data, load_raw, save_raw, save_scores
from TestSharedLib.constants import quant_support, crops, feature_count
# from data_processing_utility import save_wavefile

In [6]:
from datetime import datetime
def genDataTimeStr():
    return datetime.today().strftime('%Y-%m-%d %H:%M:%S').replace('-',"").replace(' ',"").replace(':',"");

In [7]:
model_path = "../../trained_models/step_1_base_train/tflite_model/uec_model_4Classes_20240520150535.tflite";
tflite_interpreter = tf.lite.Interpreter(model_path=model_path)

In [15]:
# model.summary()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])
tflite_interpreter.allocate_tensors();

== Input details ==
name: x.3
shape: [    1     1 30225     1]
type: <class 'numpy.float32'>

== Output details ==
name: 285
shape: [1 4]
type: <class 'numpy.float32'>


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### sound preprocessing codes

In [16]:
_inputLen = 30225
_nCrops = 2
def preprocess_setup():
    funcs = []
    funcs += [U.padding( _inputLen// 2),
              U.normalize(32768.0),
              U.multi_crop(_inputLen, _nCrops)]
              # U.single_crop(_inputLen)]
              # 

    return funcs

def preprocess_debug():
    debug_funcs = []
    debug_funcs += [U.padding( _inputLen// 2),
              # U.normalize(32768.0),]
              U.multi_crop(_inputLen, _nCrops)]
              # U.single_crop(_inputLen)]
              # 

    return debug_funcs


def preprocess(sound, funcs):
    for f in funcs:
        sound = f(sound)
    return sound;
    

def padding(pad):
    def f(sound):
        return np.pad(sound, pad, 'constant')

    return f
    

# def random_crop(size):
#     def f(sound):
#         org_size = len(sound)
#         start = random.randint(0, org_size - size)
#         return sound[start: start + size]

#     return f



In [29]:
# _funcs = preprocess_debug()
_funcs = preprocess_setup()

In [27]:
def doSoundClassification(input_wav=None, lblidx=None, channelIdx=0):
    sound = np.float32(wavio.read(input_wav).data.T[0])
    start = sound.nonzero()[0].min();
    end = sound.nonzero()[0].max();
    sound = sound[start: end + 1];
    print(f"get sound signal from {start} to {end}");
    if len(sound)> 220500:
        sound = sound[:220500]
    # if len(sound)> 30225:
    #     sound = sound[:30226]
    # sound = np.int16(preprocess(sound, _funcs));
    sound = preprocess(sound, _funcs)
    
    s_test = np.expand_dims(sound[channelIdx], axis=0)
    s_test = np.expand_dims(s_test, axis=1);
    s_test = np.expand_dims(s_test, axis=3);
    
    tflite_interpreter.set_tensor(input_details[0]['index'], s_test);
    tflite_interpreter.invoke()
    pred = tflite_interpreter.get_tensor(output_details[0]['index'])
    print(type(pred))
    # print(f"Prediction result shape:{pred.shape}\n");
    print(f"the larget value index is:{pred[0].argmax(axis=0)}")
    print(f"Prediction result: {pred}, and true label idx: {lblidx}")
    # print(f"channel of inpu_wav:{len(sound)}");

In [28]:
tmp_wav2 = "../../datasets/CurrentUse/wav_files/Single_Fold/val/positive/help_eng_71/with_white_noises/middle/man_clear_clip_help_011_71_hwn_middle.wav";
doSoundClassification(input_wav=tmp_wav2, lblidx=2, channelIdx=0)

get sound signal from 0 to 30224
<class 'numpy.ndarray'>
the larget value index is:2
Prediction result: [[6.1477418e-05 6.5865554e-04 9.1230071e-01 8.6979188e-02]], and true label idx: 2


In [31]:
CHUNK = 1024 #byte
# FORMAT = pyaudio.paInt16
CHANNELS = 1 #if sys.platform == 'darwin' else 2;
RATE = 20000
RECORD_SECONDS = 1.5
SAMPLE_SIZE = 2
FRAMES_PER_BUFFER = 64
STEAM_SAVE_DIR="./mic_record_savedir/stream_save/{}"

In [44]:
def save_wavefile(output_path, sig, sr):
    output_file = wave.Wave_write(output_path)
    output_file.setnchannels(1);
    output_file.setframerate(sr);
    output_file.setsampwidth(2) #2bytes per sample.
    # output_file.setparams(params) #nchannels, sampwidth, framerate, nframes, comptype, compname
    output_file.writeframes(array.array('h', sig.astype(np.int16)).tobytes(), )
    output_file.close()

In [49]:
def rcord_sound_by_soundfile(rec_druation = 2):
    print(f"進行{rec_druation}秒聲音錄製");
    WAVE_OUTPUT_FILENAME = "mic_test_sound_by_soundfile_{}.wav".format(genDataTimeStr());
    samplerate = 20000;#44100  # Hertz
    duration = rec_druation  # seconds
    channels = 1;
    test_wav = "./mic_record_savedir/{}".format(WAVE_OUTPUT_FILENAME);
    mydata = sd.rec(int(samplerate * duration), samplerate=samplerate,
                    channels=channels, blocking=True)
    # print(mydata)
    # save_wavefile(test_wav, mydata, 20000);
    sf.write(test_wav, mydata, samplerate);
    # scipy.io.wavfile.write(test_wav, 20000, mydata)
    return test_wav;

In [53]:
# tmp_wav = wavio.read("./mic_record_savedir/mic_test_sound_by_soundfile_20240520143508.wav").data.T[0];
# print(tmp_wav);

In [51]:
play_wav1 = None;
play_wav1 =  rcord_sound_by_soundfile(2);
print(f"living recording is {play_wav1}");
IPython.display.Audio(play_wav1)

進行2秒聲音錄製
living recording is ./mic_record_savedir/mic_test_sound_by_soundfile_20240520144342.wav


In [67]:
# doSoundClassification(input_wav=play_wav1,lblidx=1,channelIdx=1);

In [30]:
test_wav2 = "./wail_280170.wav"
test_wav3 = "";
val_wav = "../../datasets/CurrentUse/wav_files/Single_Fold/val/positive/moaning_56/man_moaning_35_1_val_56.wav"
living_record_wav = "./mic_record_savedir/mic_test_sound_by_soundfile_20240508104135.wav"
print("test from live recording wav without modification");
IPython.display.Audio(val_wav)
# doSoundClassification(input_wav=living_record_wav,lblidx=1,channelIdx=0)
# doSoundClassification(input_wav=val_wav,lblidx=1,channelIdx=0)

test from live recording wav without modification


In [24]:
# streamTestMain()

In [22]:
# p = pyaudio.PyAudio();
# #declare PyAudio stream
# stream = p.open(
# format=FORMAT,
# channels=CHANNELS,
# rate=RATE,
# input=True,
# frames_per_buffer=FRAMES_PER_BUFFER,)

# def closePyAudio():
#     stream.close();
#     p.termnate();

In [2]:
# flag = False;
# frames1 = None;
# frames2 = None;
# async def asyncReceiveAndCallDoClassification():
#     async def getSignal():
#         global frames1, frames2, flag;
#         frames1 = [];
#         print("run in getSignal");
#         while True:
#             data = stream.read(FRAMES_PER_BUFFER);
#             frames1.append(data)
#             if len(frames1) >= (RATE * RECORD_SECONDS) / FRAMES_PER_BUFFER:
#                 print("run in getSignal if");
#                 flag = True
#                 frames2 = frame1.copy();
#                 frames1 = [];
#             await asyncio.sleep(0.01)
#         return True
#     async def consumeSignal():
#         global frames1, frames2, flag;
#         print("run in consumeSignal")
#         while True:
#             if flag == True:
#                 print("run in consumeSignal flag=true")
#                 doSoundClassificationbUsingStreaming(input_data=frames2);
#                 flag = False;
#             return asyncio.gather(getSignal(), consumeSignal())

In [3]:
# await asyncReceiveAndCallDoClassification()

In [2]:
# DURATION = 2;  # seconds

# def callback(in_data, frame_count, time_info, status):
#     print("data received.....");
#     #return (in_data, pyaudio.paContinue)


# stream = p.open(format=p.get_format_from_width(2),
#                 channels=1 if sys.platform == 'darwin' else 2,
#                 rate=20000,
#                 input=True,
#                 output=True,
#                 stream_callback=callback)

# start = time.time()
# while stream.is_active() and (time.time() - start) < DURATION:
#     pass
#     # time.sleep(1)

# stream.close()
# p.terminate()

In [31]:
doSoundClassification(input_wav=living_record_wav,lblidx=1,channelIdx=0)

get sound signal from 2 to 39999
Prediction result: [[ 22 -18  39]], and true label idx: 1
